In [78]:


#First of all we need to load in the pandas library...
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt



In [6]:
from pandas_datareader import wb
matches = wb.search('SP.POP')
matches

,id,name,unit,source,sourceNote,sourceOrganization,topics
13887,SP.POP.TOTL.ICP,SP.POP.TOTL.ICP:Population,,ICP 2017,,b'',
13888,SP.POP.TOTL.ICP.ZS,SP.POP.TOTL.ICP.ZS:Population shares (World=100),,ICP 2017,,b'',


In [42]:
dat = wb.download(indicator='SP.POP.TOTL', country='ALL',start=1800, end=2022)
dat

SP.POP.TOTL
country                     year             
Africa Eastern and Southern 2021          NaN
                            2020  677243299.0
                            2019  660046272.0
                            2018  643090131.0
                            2017  626392880.0
...                                       ...
Zimbabwe                    1964    4322854.0
                            1963    4178726.0
                            1962    4039209.0
                            1961    3905038.0
                            1960    3776679.0

[16492 rows x 1 columns]

In [100]:
countries_df = wb.get_countries()
countries_df['country'] = countries_df['name']
countries_df.set_index('country', inplace=True)
countries_ds = xr.Dataset(coords=countries_df)
countries_ds.coords.update({'year': np.arange(1800, 2200), 'country': countries_ds.coords['name'].astype(str)})
countries_ds

<xarray.Dataset>
Dimensions:      (country: 299, year: 400)
Coordinates:
  * country      (country) <U80 'Aruba' ... 'Zimbabwe'
    iso3c        (country) object 'ABW' 'AFE' 'AFG' 'AFR' ... 'ZAF' 'ZMB' 'ZWE'
    iso2c        (country) object 'AW' 'ZH' 'AF' 'A9' ... 'YE' 'ZA' 'ZM' 'ZW'
    name         (country) object 'Aruba' ... 'Zimbabwe'
    region       (country) object 'Latin America & Caribbean ' ... 'Sub-Sahar...
    adminregion  (country) object '' ... 'Sub-Saharan Africa (excluding high ...
    incomeLevel  (country) object 'High income' ... 'Lower middle income'
    lendingType  (country) object 'Not classified' 'Aggregates' ... 'Blend'
    capitalCity  (country) object 'Oranjestad' '' 'Kabul' ... 'Lusaka' 'Harare'
    longitude    (country) float64 -70.02 nan 69.18 nan ... 28.19 28.29 31.07
    latitude     (country) float64 12.52 nan 34.52 nan ... -25.75 -15.4 -17.83
  * year         (year) int64 1800 1801 1802 1803 1804 ... 2196 2197 2198 2199
Data variables:
    *empty*

In [101]:
pop_ds = xr.Dataset.from_dataframe(dat)
pop_ds.coords['year'] = pop_ds.coords['year'].astype(int)
pop_ds.coords['country'] = pop_ds.coords['country'].astype(str)
pop_ds

<xarray.Dataset>
Dimensions:      (country: 266, year: 62)
Coordinates:
  * country      (country) <U52 'Afghanistan' ... 'Zimbabwe'
  * year         (year) int64 1960 1961 1962 1963 1964 ... 2018 2019 2020 2021
Data variables:
    SP.POP.TOTL  (country, year) float64 8.997e+06 9.169e+06 ... 1.486e+07 nan

In [105]:
all_ds = xr.merge([countries_ds, pop_ds])
all_ds 

<xarray.Dataset>
Dimensions:      (country: 301, year: 400)
Coordinates:
  * country      (country) <U80 'Afghanistan' 'Africa' ... 'Zambia' 'Zimbabwe'
  * year         (year) int64 1800 1801 1802 1803 1804 ... 2196 2197 2198 2199
    iso3c        (country) object 'AFG' 'AFR' 'AFE' 'AFW' ... 'YEM' 'ZMB' 'ZWE'
    iso2c        (country) object 'AF' 'A9' 'ZH' 'ZI' ... '1W' 'YE' 'ZM' 'ZW'
    name         (country) object 'Afghanistan' 'Africa' ... 'Zambia' 'Zimbabwe'
    region       (country) object 'South Asia' ... 'Sub-Saharan Africa '
    adminregion  (country) object 'South Asia' ... 'Sub-Saharan Africa (exclu...
    incomeLevel  (country) object 'Low income' ... 'Lower middle income'
    lendingType  (country) object 'IDA' 'Aggregates' ... 'IDA' 'Blend'
    capitalCity  (country) object 'Kabul' '' '' ... "Sana'a" 'Lusaka' 'Harare'
    longitude    (country) float64 69.18 nan nan nan ... nan 44.21 28.29 31.07
    latitude     (country) float64 34.52 nan nan nan ... nan 15.35 -15.4 -17.83
Data variables:
    swag         (country, year) float64 nan nan nan nan nan ... nan nan nan nan
    SP.POP.TOTL  (country, year) float64 nan nan nan nan nan ... nan nan nan nan

In [109]:
all_ds['SP.POP.TOTL'].sel(country='France', year=2015)

<xarray.DataArray 'SP.POP.TOTL' ()>
array(66548272.)
Coordinates:
    country      <U80 'France'
    year         int64 2015
    iso3c        object 'FRA'
    iso2c        object 'FR'
    name         object 'France'
    region       object 'Europe & Central Asia'
    adminregion  object ''
    incomeLevel  object 'High income'
    lendingType  object 'Not classified'
    capitalCity  object 'Paris'
    longitude    float64 2.351
    latitude     float64 48.86

In [111]:
%matplotlib qt
fig, ax = plt.subplots()
for c in countries_ds.coords['country'].to_numpy()[::3]:
    ax.plot(all_ds['year'], all_ds['SP.POP.TOTL'].sel(country=c), label=c)
ax.legend()
plt.show()

In [93]:
countries_ds['swag'].sel(country=c)

<xarray.DataArray 'swag' (year: 400)>
array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])
Coordinates:
    country      <U7 'Vanuatu'
  * year         (year) int64 1800 1801 1802 1803 1804 ... 2196 2197 2198 2199
    iso3c        object 'VUT'
    iso2c        object 'VU'
    name         object 'Vanuatu'
    region       object 'East Asia & Pacific'
    adminregion  object 'East Asia & Pacific (excluding high income)'
    incomeLevel  object 'Lower middle income'
    lendingType  object 'IDA'
    capitalCity  object 'Port-Vila'
    longitude    float64 168.3
    latitude     float64 -17.74

In [94]:
pop_ds

<xarray.Dataset>
Dimensions:      (country: 266, year: 62)
Coordinates:
  * country      (country) object 'Afghanistan' ... 'Zimbabwe'
  * year         (year) object '1960' '1961' '1962' ... '2019' '2020' '2021'
Data variables:
    SP.POP.TOTL  (country, year) float64 8.997e+06 9.169e+06 ... 1.486e+07 nan